For vanilla cnn

In [1]:
import numpy as np
import tensorflow as tf
import h5py

In [27]:
A01T = h5py.File("./project_datasets/A01T_slice.mat", 'r')
X = np.copy(A01T["image"])
y = np.copy(A01T["type"])
y = y[0, 0:X.shape[0]:1]
y = np.asarray(y, dtype=np.int32)

In [28]:
print("shape of X:", X.shape)
print("shape of y:", y.shape)

shape of X: (288, 25, 1000)
shape of y: (288,)


In [39]:
print(y)

[772 771 770 769 769 770 771 772 770 771 769 769 769 772 770 770 769 769
 771 769 770 772 772 771 769 772 772 770 772 772 770 769 770 771 771 771
 772 771 769 772 770 771 770 771 772 770 771 769 769 769 772 770 769 771
 769 771 770 772 769 771 771 769 771 770 772 772 772 771 769 772 770 772
 770 769 771 770 769 771 771 769 771 772 772 770 769 770 772 770 772 771
 770 770 770 771 772 769 770 772 769 771 771 772 769 769 771 770 772 772
 772 770 769 771 770 772 769 772 771 770 772 772 769 770 770 771 772 770
 769 769 772 770 769 771 770 770 771 769 772 771 771 771 771 769 770 769
 770 769 769 771 771 770 771 772 769 772 769 769 770 772 771 770 772 771
 772 771 772 770 770 772 769 770 770 770 771 772 769 772 769 771 769 772
 769 771 769 770 771 771 772 769 770 772 770 771 771 769 772 770 772 769
 769 771 771 770 772 770 770 769 770 772 772 770 770 770 770 772 772 771
 772 769 770 771 770 769 772 769 772 769 769 769 769 771 771 772 770 771
 771 771 772 771 769 771 769 772 770 772 772 771 77

Build a 1 layer cnn with tf

In [37]:
x_in = tf.placeholder(tf.float32, [None, 25, 1000], name="input_x")
y_real = tf.placeholder(tf.int32, [None], name="real_y")
input_layer = tf.reshape(x_in, [-1, 25, 1000, 1], name="reshaped_x")

conv1 = tf.layers.conv2d(
    inputs=input_layer,
    filters=32,
    kernel_size=[3, 3],
    padding="same",
    activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

conv2 = tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[3, 3],
    padding="same",
    activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

pool2_flat = tf.reshape(pool2, [-1, (25*1000*64)//16])
aff3 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

logits = tf.layers.dense(inputs=aff3, units=4)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y_real, logits=logits)
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [38]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1):
        inputs = {x_in: X, y_real: y}
        loss, _ = sess.run([loss, optimizer], feed_dict=inputs)
        print("loss is:", loss)

InvalidArgumentError: Input to reshape is a tensor with 27648000 values, but the requested shape requires a multiple of 100000
	 [[Node: Reshape_2 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](max_pooling2d_6/MaxPool, Reshape_2/shape)]]

Caused by op 'Reshape_2', defined at:
  File "/Users/charliezjw/anaconda3/envs/tfpy36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/charliezjw/anaconda3/envs/tfpy36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/charliezjw/anaconda3/envs/tfpy36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/charliezjw/anaconda3/envs/tfpy36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/charliezjw/anaconda3/envs/tfpy36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in 